In [2]:
import os
iskaggle = os.environ.get('KAGGLE_KERNEL_RUN_TYPE', '')

In [3]:
if iskaggle:
    from pathlib import Path
    path = Path('/kaggle/input/imdb-dataset-of-50k-movie-reviews')
    ! pip install -q datasets

In [4]:
from pathlib import Path
path = Path('/kaggle/input/imdb-dataset-of-50k-movie-reviews')
import pandas as pd
df = pd.read_csv('/kaggle/input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv')
#df=df.head(8000)
df

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
...,...,...
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative


In [5]:
df.describe(include='object')

,review,sentiment
count,50000,50000
unique,49582,2
top,Loved today's show!!! It was a variety and not...,positive
freq,5,25000


In [6]:
df.loc[df['sentiment']=='positive','sentiment']=1
df.loc[df['sentiment']=='negative','sentiment']=0
df['sentiment'].apply(float)

0        1.0
1        1.0
2        1.0
3        0.0
4        1.0
        ... 
49995    1.0
49996    0.0
49997    0.0
49998    0.0
49999    0.0
Name: sentiment, Length: 50000, dtype: float64

In [7]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [8]:
from datasets import Dataset,DatasetDict

ds = Dataset.from_pandas(df)
ds

Dataset({
    features: ['review', 'sentiment'],
    num_rows: 50000
})

In [9]:
model_nm ='cardiffnlp/twitter-xlm-roberta-base-sentiment'


In [10]:
from transformers import AutoModelForSequenceClassification,AutoTokenizer
tokz = AutoTokenizer.from_pretrained(model_nm,model_max_length=256)
max_length = tokz.model_max_length

In [11]:
def tok_func(x): return tokz(x["review"],truncation=True,
  max_length=max_length)

In [12]:
tok_ds = ds.map(tok_func, batched=True)

  0%|          | 0/50 [00:00<?, ?ba/s]

In [13]:
tok_ds

Dataset({
    features: ['review', 'sentiment', 'input_ids', 'attention_mask'],
    num_rows: 50000
})

In [14]:
tok_ds[5]['review'], tok_ds[5]['input_ids']


('Probably my all-time favorite movie, a story of selflessness, sacrifice and dedication to a noble cause, but it\'s not preachy or boring. It just never gets old, despite my having seen it some 15 or more times in the last 25 years. Paul Lukas\' performance brings tears to my eyes, and Bette Davis, in one of her very few truly sympathetic roles, is a delight. The kids are, as grandma says, more like "dressed-up midgets" than children, but that only makes them more fun to watch. And the mother\'s slow awakening to what\'s happening in the world and under her own roof is believable and startling. If I had a dozen thumbs, they\'d all be "up" for this movie.',
 [0,
  1250,
  17105,
  538,
  759,
  756,
  9,
  6032,
  40304,
  14277,
  4,
  10,
  13765,
  111,
  15970,
  9393,
  7432,
  4,
  209201,
  136,
  176762,
  191,
  47,
  10,
  110,
  2661,
  22304,
  4,
  1284,
  442,
  25,
  7,
  959,
  19542,
  7668,
  707,
  3017,
  214,
  5,
  1650,
  1660,
  8306,
  62163,
  10332,
  4,
  8,

In [15]:
tokz.vocab['.']

5

In [16]:
tok_ds = tok_ds.rename_columns({'sentiment':'labels'})
tok_ds

Dataset({
    features: ['review', 'labels', 'input_ids', 'attention_mask'],
    num_rows: 50000
})

In [17]:
dds = tok_ds.train_test_split(0.25, seed=42)
dds

DatasetDict({
    train: Dataset({
        features: ['review', 'labels', 'input_ids', 'attention_mask'],
        num_rows: 37500
    })
    test: Dataset({
        features: ['review', 'labels', 'input_ids', 'attention_mask'],
        num_rows: 12500
    })
})

In [18]:
from transformers import TrainingArguments,Trainer

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [19]:
pip install evaluate

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.8 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [20]:
import numpy as np
import evaluate
def compute_metrics(eval_pred):
    metrics = dict()
    accuracy_metric = evaluate.load("accuracy")
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    metrics.update(accuracy_metric.compute(references=labels, predictions=predictions))
    return metrics

In [21]:
bs = 32
epochs = 3
lr = 8e-5

In [22]:
args = TrainingArguments('outputs', learning_rate=lr, warmup_ratio=0.1, lr_scheduler_type='cosine', fp16=True,
    evaluation_strategy="epoch", per_device_train_batch_size=bs, per_device_eval_batch_size=bs*2,
    num_train_epochs=epochs, weight_decay=0.01, report_to='none')

In [23]:
model = AutoModelForSequenceClassification.from_pretrained(model_nm, num_labels=2,ignore_mismatched_sizes=True)
trainer = Trainer(model, args, train_dataset=dds['train'], eval_dataset=dds['test'],
                  tokenizer=tokz, compute_metrics=compute_metrics,)

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at cardiffnlp/twitter-xlm-roberta-base-sentiment and are newly initialized because the shapes did not match:
- classifier.out_proj.weight: found shape torch.Size([3, 768]) in the checkpoint and torch.Size([2, 768]) in the model instantiated
- classifier.out_proj.bias: found shape torch.Size([3]) in the checkpoint and torch.Size([2]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [24]:
tok_ds = tok_ds.remove_columns('review')

In [25]:
tok_ds

Dataset({
    features: ['labels', 'input_ids', 'attention_mask'],
    num_rows: 50000
})

In [26]:
tokz.model_max_length

256

In [27]:
trainer.train();

You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss,Accuracy
1,0.331100,0.225379,0.910560
2,0.204900,0.221683,0.919760
3,0.118900,0.280739,0.922320


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead u

In [28]:
trainer.save_model('nlp.ptc')

In [29]:
trainer.push_to_hub("Bittar/imdb")

ValueError: Token is required (write-access action) but no token found. You need to provide a token or be logged in to Hugging Face with `huggingface-cli login` or `huggingface_hub.login`. See https://huggingface.co/settings/tokens.

In [31]:
lst=['i hated the movie!']
df=pd.DataFrame(lst,columns=['review'])
tok_ds2=Dataset.from_pandas(df).map(tok_func, batched=True)

  0%|          | 0/1 [00:00<?, ?ba/s]

In [34]:
preds=trainer.predict(tok_ds2).predictions.astype(float)
preds

array([[ 3.85053778, -3.50595212]])

In [35]:
preds.argmax()

0

In [32]:
!zip -r file2.zip /kaggle/working/nlp.ptc

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
  adding: kaggle/working/nlp.ptc/ (stored 0%)
  adding: kaggle/working/nlp.ptc/tokenizer_config.json (deflated 49%)
  adding: kaggle/working/nlp.ptc/tokenizer.json (deflated 76%)
  adding: kaggle/working/nlp.ptc/pytorch_model.bin (deflated 7%)
  adding: kaggle/working/nlp.ptc/sentencepiece.bpe.model (deflated 49%)
  adding: kaggle/working/nlp.ptc/special_tokens_map.json (deflated 57%)
  adding: kaggle/working/nlp.ptc/training_args.bin (deflated 48%)
  adding: kaggle/working/nlp.ptc/config.json (deflated 49%)


In [36]:
import os
os.chdir(r'/kaggle/working/nlp.ptc')

!tar -czf nlp.tar.gz images_out/Landscapes

from IPython.display import FileLink

FileLink(r'Landscapes.tar.gz')

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
tar: images_out/Landscapes: Cannot stat: No such file or directory
tar: Exiting with failure status due to previous errors


/kaggle/working/nlp.ptc/Landscapes.tar.gz